## Exercise 54 - MOTIF

In [1]:
from graphframes import GraphFrame

inputPathVertexes = "/data/students/bigdata-01QYD/ex_data/Ex54/data/vertexes.csv"
inputPathEdges = "/data/students/bigdata-01QYD/ex_data/Ex54/data/edges.csv"
outputPath = "resOut_ex54/"

In [2]:
verticesDF = spark.read.load(inputPathVertexes, format="csv", header=True, inferSchema=True)
edgesDF = spark.read.load(inputPathEdges, format="csv", header=True, inferSchema=True)

In [3]:
verticesDF.printSchema()
verticesDF.show()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)

+---+-----+---+
| id| name|age|
+---+-----+---+
| u1|Alice| 34|
| u2|  Bob| 36|
| u3| John| 30|
| u4|David| 29|
| u5| Paul| 32|
| u6| Adel| 36|
| u7| Eddy| 60|
+---+-----+---+



In [4]:
edgesDF.printSchema()
edgesDF.show()

root
 |-- src: string (nullable = true)
 |-- dst: string (nullable = true)
 |-- linktype: string (nullable = true)

+---+---+--------+
|src|dst|linktype|
+---+---+--------+
| u1| u2|  friend|
| u1| u5|  friend|
| u2| u3|  follow|
| u3| u2|  follow|
| u4| u1|  friend|
| u4| u5|  friend|
| u5| u1|  friend|
| u5| u4|  friend|
| u5| u6|  follow|
| u6| u3|  follow|
| u7| u6|  follow|
+---+---+--------+



Consider :
* The pairs of users Ux, Uy such that 
    * Ux is friend of Uy (link “friend” from Ux to Uy)
    * Uy is not friend of Uy (no link “friend” from Uy to Ux)
* One pair Ux,Uy per line
* Format: idUx, idUy
* Use the CSV format to store the result

In [14]:
filteredEdges = edgesDF.filter("linktype='friend' ")
g = GraphFrame(verticesDF, filteredEdges)

In [16]:
# since we're dealing with all "friend" edges, 
# we can use anonymous edges
selectedPaths = g.find("(UserX)-[]->(UserY); !(UserY)-[]->(UserX)")
selectedPaths.show()

+---------------+---------------+
|          UserX|          UserY|
+---------------+---------------+
|[u4, David, 29]|[u1, Alice, 34]|
|[u1, Alice, 34]|  [u2, Bob, 36]|
+---------------+---------------+



In [17]:
# extract just the IDs
cleanedSelectedPaths = selectedPaths.selectExpr("UserX.id as idUx", "UserY.id as idUy")
cleanedSelectedPaths.show()

+----+----+
|idUx|idUy|
+----+----+
|  u4|  u1|
|  u1|  u2|
+----+----+



In [18]:
cleanedSelectedPaths.write.csv(outputPath, header=True)